In [ ]:
"""
you ("the sky") and why your map looks mostly blank ("you sure ask a lot of questions") and hang on did you just 


"""

def get_something(s: str) -> int:
    return
    
with open("input.txt") as f:
    print(get_something(f.read()))
    
#sum is 

In [ ]:
"""
you ("the sky") and why your map looks mostly blank ("you sure ask a lot of questions") and hang on did you just 


"""

def get_something(s: str) -> int:
    return
    
with open("input.txt") as f:
    print(get_something(f.read()))
    
#sum is 